In [4]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

file_names = [
    'dishwasher_53.csv',
    'boiler_226.csv',
    'air_purifier_293.csv',
    'sound_system_252.csv',
    '3D_printer_29.csv',
    'coffee_54.csv',
    'phone_charger_282.csv',
    'fridge_207.csv',
    'radiator_309.csv',
    'dehumidifier_310.csv',
    'fridge_317.csv',
    'micro_wave_oven_314.csv',
    'laptop_289.csv',
    'tv_290.csv',
    'vacuum_236.csv',
    'screen_302.csv',
    'dehumidifier_322.csv',
    'solar_panel_325.csv',
    'screen_146.csv',
    'washing_machine_157.csv',
    'fan_215.csv',
    'air_conditioner_222.csv',
    'laptop_64.csv',
    'coffee_37.csv',
    'washing_machine_52.csv',
    'computer_44.csv',
    'boiler_233.csv',
    'micro_wave_oven_147.csv',
    'printer_286.csv',
    'fridge_284.csv',
    'coffee_97.csv',
    'fridge_98.csv',
    'washing_machine_135.csv',
    'internet_router_131.csv',
    'dryer_219.csv',
    'boiler_217.csv',
    'washing_machine_218.csv',
    'freezer_249.csv'
]

# Directorio donde se encuentran los archivos CSV
csv_directory = './archive/'

# Crear un DataFrame vacío para almacenar todos los datos
df = pd.DataFrame()

# Leer cada archivo CSV y agregarlo al DataFrame
for file_name in file_names:
    file_path = os.path.join(csv_directory, file_name)
    df_temp = pd.read_csv(file_path)
    
    # Seleccionar una muestra del 5% de los datos
    df_temp = df_temp.sample(frac=0.05)
    
    df = pd.concat([df, df_temp])

# Convertir la columna de timestamp en datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Establecer la columna de timestamp como el índice del dataframe
df = df.set_index('timestamp')

# Normalizar los datos de potencia
df['power'] = (df['power'] - df['power'].mean()) / df['power'].std()

# Crear secuencias de datos para entrenar la red LSTM
sequence_length = 10
num_features = 1

data = []
for i in range(len(df) - sequence_length):
    data.append(df.iloc[i:i+sequence_length])
data = np.array(data)

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(0.8 * len(data))
x_train = data[:train_size, :-1]
y_train = data[:train_size, -1]
x_test = data[train_size:, :-1]
y_test = data[train_size:, -1]

# Crear la red LSTM
model2 = Sequential()
model2.add(LSTM(50, input_shape=(sequence_length-1, num_features)))
model2.add(Dense(num_features))

# Definir el optimizador Adam con una tasa de aprendizaje inicial
opt = Adam(learning_rate=0.001)

# Compilar el modelo con el optimizador Adam
model2.compile(optimizer=opt, loss='mse')

# Entrenar la red LSTM
model2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
120510/120510 [==============================] - 919s 8ms/step - loss: 0.5293 - val_loss: 0.5748
Epoch 2/10
120510/120510 [==============================] - 1485s 12ms/step - loss: 0.5265 - val_loss: 0.5735
Epoch 3/10
120510/120510 [==============================] - 826s 7ms/step - loss: 0.5257 - val_loss: 0.5733
Epoch 4/10
120510/120510 [==============================] - 836s 7ms/step - loss: 0.5255 - val_loss: 0.5710
Epoch 5/10
120510/120510 [==============================] - 829s 7ms/step - loss: 0.5253 - val_loss: 0.5728
Epoch 6/10
120510/120510 [==============================] - 826s 7ms/step - loss: 0.5252 - val_loss: 0.5724
Epoch 7/10
120510/120510 [==============================] - 903s 7ms/step - loss: 0.5250 - val_loss: 0.5729
Epoch 8/10
120510/120510 [==============================] - 1156s 10ms/step - loss: 0.5252 - val_loss: 0.5733
Epoch 9/10
120510/120510 [==============================] - 824s 7ms/step - loss: 0.5248 - val_loss: 0.5734
Epoch 10/10
120510/12051

Predicciones

In [5]:
# Hacer predicciones en el conjunto de prueba
y_pred = model2.predict(x_test)

#print("Predicción: ", y_pred)

# Calcular el error cuadrático medio (MSE)
mse = np.mean((y_pred - y_test)**2)
print(f'MSE: {mse}')


threshold = 0.1
accuracy = np.mean(np.abs(y_pred - y_test) <= threshold * np.abs(y_test))
print(f'Accuracy: {accuracy}')

30128/30128 [==============================] - 83s 3ms/step
MSE: 0.5722263542006003
Accuracy: 0.011109036594473487
